In [52]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from scipy.interpolate import interp2d
#import math


### 1. Load Station data and AQI data into Pandas

### 1.1 Load Raw Data

In [66]:
df_stations = pd.read_csv(r'D:\Tommy\+WeatherData\china_aqi_map\stations_20220213.csv')
df_aqis = pd.read_csv(r'D:\Tommy\+WeatherData\china_aqi_map\china_sites_20220520.csv')

### 1.2 Rename the column names into English

In [67]:
df_stations.drop(['对照点'], axis=1, inplace = True)
df_stations.rename(columns = {'监测点编码':'station_id', '监测点名称':'loc_name', '城市':'city', '经度':'lon', '纬度':'lat'}, inplace = True)
print(df_stations)

     station_id loc_name  city       lon       lat
0         1001A     万寿西宫    北京  116.3621   39.8784
1         1002A  定陵(对照点)    北京  116.2202   40.2915
2         1003A       东四    北京  116.4174   39.9289
3         1004A       天坛    北京  116.4072   39.8863
4         1005A      农展馆    北京   116.462   39.9365
...         ...      ...   ...       ...       ...
2021      3734A     理想之城    东营    118.35     37.26
2022      3860A   皇山花园小区    临沂   118.392     35.03
2023      3861A  寒亭区实验一小    潍坊  119.2041   36.7699
2024      3866A      三高中    营口  122.2469  40.66302
2025      3867A  平安区公安北苑  海东地区  102.0738   36.5056

[2026 rows x 5 columns]


In [68]:
print(df_aqis)

         date  hour       type  1001A  1002A  1003A  1004A  1005A  1006A  \
0    20220520     0        AQI   84.0   31.0   95.0   80.0   87.0   83.0   
1    20220520     0      PM2.5   47.0   12.0   65.0   45.0   44.0   48.0   
2    20220520     0  PM2.5_24h   37.0   26.0   44.0   37.0   38.0   39.0   
3    20220520     0       PM10  117.0   26.0  139.0  110.0  124.0  115.0   
4    20220520     0   PM10_24h   79.0   55.0   90.0   77.0   91.0   83.0   
..        ...   ...        ...    ...    ...    ...    ...    ...    ...   
355  20220520    23     O3_24h  241.0  210.0  240.0  242.0  251.0  248.0   
356  20220520    23      O3_8h  208.0  175.0  213.0  206.0  222.0  218.0   
357  20220520    23  O3_8h_24h  234.0  195.0  233.0  233.0  240.0  236.0   
358  20220520    23         CO    0.7    0.5    0.7    0.7    0.6    0.7   
359  20220520    23     CO_24h    0.8    0.3    0.8    0.7    0.9    0.7   

     1007A  ...  3713A  3715A  3719A  3726A  3728A  3734A  3860A  3861A  \
0     77.0  

### 1.3 Select ONE row by hour and type

In [69]:
df1 = df_aqis[(df_aqis['hour'] == 0) & (df_aqis['type'] == 'PM2.5')]
print(df1)

       date  hour   type  1001A  1002A  1003A  1004A  1005A  1006A  1007A  \
1  20220520     0  PM2.5   47.0   12.0   65.0   45.0   44.0   48.0   39.0   

   ...  3713A  3715A  3719A  3726A  3728A  3734A  3860A  3861A  3866A  3867A  
1  ...   30.0   41.0    3.0    6.0   18.0   51.0   32.0   39.0   22.0   39.0  

[1 rows x 2029 columns]


### 1.4 Drop the stations don't have data

In [70]:
df2 = df1.dropna(axis=1)
print(df2)

       date  hour   type  1001A  1002A  1003A  1004A  1005A  1006A  1007A  \
1  20220520     0  PM2.5   47.0   12.0   65.0   45.0   44.0   48.0   39.0   

   ...  3713A  3715A  3719A  3726A  3728A  3734A  3860A  3861A  3866A  3867A  
1  ...   30.0   41.0    3.0    6.0   18.0   51.0   32.0   39.0   22.0   39.0  

[1 rows x 1705 columns]


### 1.5 Swap Columns and Rows (AQI data are stored in columns)

In [71]:
df3 = df2.transpose()
print(df3)

              1
date   20220520
hour          0
type      PM2.5
1001A      47.0
1002A      12.0
...         ...
3734A      51.0
3860A      32.0
3861A      39.0
3866A      22.0
3867A      39.0

[1705 rows x 1 columns]


### 1.6 Reset Index and change the column names

In [72]:
df4 = df3.reset_index()
print (df4)

      index         1
0      date  20220520
1      hour         0
2      type     PM2.5
3     1001A      47.0
4     1002A      12.0
...     ...       ...
1700  3734A      51.0
1701  3860A      32.0
1702  3861A      39.0
1703  3866A      22.0
1704  3867A      39.0

[1705 rows x 2 columns]


In [79]:
df4.rename(columns = {'index':'s_id', 1:'value'}, inplace = True)
print (df4)

       s_id     value
0      date  20220520
1      hour         0
2      type     PM2.5
3     1001A      47.0
4     1002A      12.0
...     ...       ...
1700  3734A      51.0
1701  3860A      32.0
1702  3861A      39.0
1703  3866A      22.0
1704  3867A      39.0

[1705 rows x 2 columns]


### 1.7 Remove extra first 3 rows (generated by column row swap) 

In [80]:
df_values = df4[3:]
print(df_values)

       s_id value
3     1001A  47.0
4     1002A  12.0
5     1003A  65.0
6     1004A  45.0
7     1005A  44.0
...     ...   ...
1700  3734A  51.0
1701  3860A  32.0
1702  3861A  39.0
1703  3866A  22.0
1704  3867A  39.0

[1702 rows x 2 columns]


### 1.8 Join Stations and AQI data

In [81]:
print(df_stations)

     station_id loc_name  city       lon       lat
0         1001A     万寿西宫    北京  116.3621   39.8784
1         1002A  定陵(对照点)    北京  116.2202   40.2915
2         1003A       东四    北京  116.4174   39.9289
3         1004A       天坛    北京  116.4072   39.8863
4         1005A      农展馆    北京   116.462   39.9365
...         ...      ...   ...       ...       ...
2021      3734A     理想之城    东营    118.35     37.26
2022      3860A   皇山花园小区    临沂   118.392     35.03
2023      3861A  寒亭区实验一小    潍坊  119.2041   36.7699
2024      3866A      三高中    营口  122.2469  40.66302
2025      3867A  平安区公安北苑  海东地区  102.0738   36.5056

[2026 rows x 5 columns]


In [82]:
print(df_values)

       s_id value
3     1001A  47.0
4     1002A  12.0
5     1003A  65.0
6     1004A  45.0
7     1005A  44.0
...     ...   ...
1700  3734A  51.0
1701  3860A  32.0
1702  3861A  39.0
1703  3866A  22.0
1704  3867A  39.0

[1702 rows x 2 columns]


In [83]:
df_ret = df_stations.set_index('station_id').join(df_values.set_index('s_id'), lsuffix="_left", rsuffix="_right", how='right')
print(df_ret)

      loc_name  city       lon       lat value
s_id                                          
1001A     万寿西宫    北京  116.3621   39.8784  47.0
1002A  定陵(对照点)    北京  116.2202   40.2915  12.0
1003A       东四    北京  116.4174   39.9289  65.0
1004A       天坛    北京  116.4072   39.8863  45.0
1005A      农展馆    北京   116.462   39.9365  44.0
...        ...   ...       ...       ...   ...
3734A     理想之城    东营    118.35     37.26  51.0
3860A   皇山花园小区    临沂   118.392     35.03  32.0
3861A  寒亭区实验一小    潍坊  119.2041   36.7699  39.0
3866A      三高中    营口  122.2469  40.66302  22.0
3867A  平安区公安北苑  海东地区  102.0738   36.5056  39.0

[1702 rows x 5 columns]


### 1.9 Save result to csv file

In [85]:
df_ret.to_csv(r'D:\Tommy\+WeatherData\china_aqi_map\aqi_all.csv')